<h2>PROBLEM STATEMENT</h2>
<h2>EVALUATION MATRIX</h2>


In [1]:
#IMPORTING REQUIRED LIBRARIES
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
import math

from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

import gc
gc.enable()


import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
#DATASET VIEW
path1="/kaggle/input/ashrae-energy-prediction/"
data_files=list(os.listdir(path1))
df_files=pd.DataFrame(data_files,columns=['File_Name'])
df_files['Size_in_MB']=df_files.File_Name.apply(lambda x:round(os.stat(path1+x).st_size/(1024*1024),2))
df_files

,File_Name,Size_in_MB
0,building_metadata.csv,0.04
1,weather_train.csv,7.10
2,train.csv,647.18
3,sample_submission.csv,426.83
4,test.csv,1394.71
5,weather_test.csv,14.10


<h3>Data Reading Strategy</h3>
1. Will read only train, weather_train and building_metadata to start with.
2. As test set is big in size will be reading it after model training.
3. Not try to reduce datasize as per my understanding by reducing datasize sometimes we loose important information. 

In [3]:
%%time
#READING TRAIN DATASET
df_train=pd.read_csv(path1+'train.csv')
df_weather_train=pd.read_csv(path1+'weather_train.csv')
df_building_metadata=pd.read_csv(path1+'building_metadata.csv')

CPU times: user 10.2 s, sys: 1.9 s, total: 12.1 s
Wall time: 12.1 s


In [4]:
#All FUNCTIONS

#FUNCTION FOR PROVIDING FEATURE SUMMARY
def feature_summary(df_fa):
    print('DataFrame shape')
    print('rows:',df_fa.shape[0])
    print('cols:',df_fa.shape[1])
    col_list=['Null','Unique_Count','Data_type','Max/Min','Mean','Std','Skewness','Sample_values']
    df=pd.DataFrame(index=df_fa.columns,columns=col_list)
    df['Null']=list([len(df_fa[col][df_fa[col].isnull()]) for i,col in enumerate(df_fa.columns)])
    #df['%_Null']=list([len(df_fa[col][df_fa[col].isnull()])/df_fa.shape[0]*100 for i,col in enumerate(df_fa.columns)])
    df['Unique_Count']=list([len(df_fa[col].unique()) for i,col in enumerate(df_fa.columns)])
    df['Data_type']=list([df_fa[col].dtype for i,col in enumerate(df_fa.columns)])
    for i,col in enumerate(df_fa.columns):
        if 'float' in str(df_fa[col].dtype) or 'int' in str(df_fa[col].dtype):
            df.at[col,'Max/Min']=str(round(df_fa[col].max(),2))+'/'+str(round(df_fa[col].min(),2))
            df.at[col,'Mean']=df_fa[col].mean()
            df.at[col,'Std']=df_fa[col].std()
            df.at[col,'Skewness']=df_fa[col].skew()
        elif 'datetime64[ns]' in str(df_fa[col].dtype):
            df.at[col,'Max/Min']=str(df_fa[col].max())+'/'+str(df_fa[col].min())
        df.at[col,'Sample_values']=list(df_fa[col].unique())
           
    return(df.fillna('-'))

#FUNCTION FOR READING DICTIONARY ITEMS AND HANDLING KEYERROR
def get_val(x,col):
    try:
        y=x[col]
    except:
        y=np.nan
    return(y)

#FUNCTION FOR CALCULATING RSME
def rsme(y,pred):
    return(mean_squared_error(y,pred)**0.5)

In [5]:
#FEATURE SUMMARY FOR TRAIN DATASET
feature_summary(df_train)

DataFrame shape
rows: 20216100
cols: 4


,Null,Unique_Count,Data_type,Max/Min,Mean,Std,Skewness,Sample_values
building_id,0,1449,int64,1448/0,799.278,426.913,-0.312457,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
meter,0,4,int64,3/0,0.662441,0.930992,1.182,"[0, 3, 1, 2]"
timestamp,0,8784,object,-,-,-,-,"[2016-01-01 00:00:00, 2016-01-01 01:00:00, 201..."
meter_reading,0,1688175,float64,21904700.0/0.0,2117.12,153236,104.811,"[0.0, 53.2397, 43.0013, 52.4206, 23.3036, 0.37..."


In [6]:
#CONVERTING timestamp TO DATATIME FIELD
df_train['timestamp']=pd.to_datetime(df_train['timestamp'])

In [7]:
#FEATURE SUMMARY FOR TRAIN DATASET AFTER CONVERTING timestamp TO DATETIME FORMAT
feature_summary(df_train)

DataFrame shape
rows: 20216100
cols: 4


,Null,Unique_Count,Data_type,Max/Min,Mean,Std,Skewness,Sample_values
building_id,0,1449,int64,1448/0,799.278,426.913,-0.312457,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
meter,0,4,int64,3/0,0.662441,0.930992,1.182,"[0, 3, 1, 2]"
timestamp,0,8784,datetime64[ns],2016-12-31 23:00:00/2016-01-01 00:00:00,-,-,-,"[2016-01-01T00:00:00.000000000, 2016-01-01T01:..."
meter_reading,0,1688175,float64,21904700.0/0.0,2117.12,153236,104.811,"[0.0, 53.2397, 43.0013, 52.4206, 23.3036, 0.37..."


<h3>UNDERSTANDING TRAIN FEATURES</h3>
<table align=left >
    <tr>
        <th  bgcolor="cyan"><b>FEATURE NAME</b></th>
        <th  bgcolor="cyan"><b>FEATURE DESCRIPTION</b></th>
        <th  bgcolor="cyan"><b>ADDITIONAL INFORMATION</b></th> 
    </tr>
    <tr>
        <td>building_id</td>
        <td>a unique buildig identifier</td>
        <td>Data contains 1449 unique builidings</td>
    </tr>
    <tr>
        <td>meter</td>
        <td>meter type</td>
        <td>four different meter types. we can create models for each meter type</td>
    </tr>
    <tr>
        <td>timestamp</td>
        <td>timestamp for meter reading</td>
        <td>we have readings for year 2016, starting from 2016-01-01 to 2016-12-31</td>
    </tr>
    <tr>
        <td>meter_reading</td>
        <td>target feature</td>
        <td>target data is highly skewed</td>
    </tr>
</table>

In [8]:
#FEATURE SUMMARY FOR WEATHER TRAIN DATASET
feature_summary(df_weather_train)

DataFrame shape
rows: 139773
cols: 9


,Null,Unique_Count,Data_type,Max/Min,Mean,Std,Skewness,Sample_values
site_id,0,16,int64,15/0,7.47898,4.60474,0.0031113,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
timestamp,0,8784,object,-,-,-,-,"[2016-01-01 00:00:00, 2016-01-01 01:00:00, 201..."
air_temperature,55,620,float64,47.2/-28.9,14.4181,10.6266,-0.318658,"[25.0, 24.4, 22.8, 21.1, 20.0, 19.4, 20.6, 18...."
cloud_coverage,69173,11,float64,9.0/0.0,2.14931,2.59915,0.978444,"[6.0, nan, 2.0, 4.0, 8.0, 0.0, 9.0, 7.0, 5.0, ..."
dew_temperature,113,523,float64,26.1/-35.0,7.35016,9.79023,-0.450697,"[20.0, 21.1, 20.6, 19.4, 18.9, 18.3, 17.8, 17...."
precip_depth_1_hr,50289,129,float64,343.0/-1.0,0.983047,8.46368,18.2862,"[nan, -1.0, 0.0, 3.0, 5.0, 8.0, 2.0, 10.0, 20...."
sea_level_pressure,10618,710,float64,1045.5/968.2,1016.16,7.62968,-0.415321,"[1019.7, 1020.2, 1020.1, 1020.0, nan, 1019.4, ..."
wind_direction,6268,44,float64,360.0/0.0,180.527,111.524,-0.168956,"[0.0, 70.0, 250.0, 210.0, 290.0, 230.0, 200.0,..."
wind_speed,304,59,float64,19.0/0.0,3.56053,2.33587,0.829448,"[0.0, 1.5, 2.6, 3.1, 4.1, 3.6, 5.1, 4.6, 2.1, ..."


In [9]:
#CONVERTING timestamp TO DATATIME FIELD IN WEATHER TRAIN DATASET
df_weather_train['timestamp']=pd.to_datetime(df_weather_train['timestamp'])

In [10]:
#FEATURE SUMMARY FOR WEATHER TRAIN DATASET AFTER CONVERTING timestamp TO DATETIME FORMAT
feature_summary(df_weather_train)

DataFrame shape
rows: 139773
cols: 9


,Null,Unique_Count,Data_type,Max/Min,Mean,Std,Skewness,Sample_values
site_id,0,16,int64,15/0,7.47898,4.60474,0.0031113,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
timestamp,0,8784,datetime64[ns],2016-12-31 23:00:00/2016-01-01 00:00:00,-,-,-,"[2016-01-01T00:00:00.000000000, 2016-01-01T01:..."
air_temperature,55,620,float64,47.2/-28.9,14.4181,10.6266,-0.318658,"[25.0, 24.4, 22.8, 21.1, 20.0, 19.4, 20.6, 18...."
cloud_coverage,69173,11,float64,9.0/0.0,2.14931,2.59915,0.978444,"[6.0, nan, 2.0, 4.0, 8.0, 0.0, 9.0, 7.0, 5.0, ..."
dew_temperature,113,523,float64,26.1/-35.0,7.35016,9.79023,-0.450697,"[20.0, 21.1, 20.6, 19.4, 18.9, 18.3, 17.8, 17...."
precip_depth_1_hr,50289,129,float64,343.0/-1.0,0.983047,8.46368,18.2862,"[nan, -1.0, 0.0, 3.0, 5.0, 8.0, 2.0, 10.0, 20...."
sea_level_pressure,10618,710,float64,1045.5/968.2,1016.16,7.62968,-0.415321,"[1019.7, 1020.2, 1020.1, 1020.0, nan, 1019.4, ..."
wind_direction,6268,44,float64,360.0/0.0,180.527,111.524,-0.168956,"[0.0, 70.0, 250.0, 210.0, 290.0, 230.0, 200.0,..."
wind_speed,304,59,float64,19.0/0.0,3.56053,2.33587,0.829448,"[0.0, 1.5, 2.6, 3.1, 4.1, 3.6, 5.1, 4.6, 2.1, ..."


<h3>UNDERSTANDING WEATHER TRAIN FEATURES</h3>
<table align=left >
    <tr>
        <th  bgcolor="cyan"><b>FEATURE NAME</b></th>
        <th  bgcolor="cyan"><b>FEATURE DESCRIPTION</b></th>
        <th  bgcolor="cyan"><b>ADDITIONAL INFORMATION</b></th> 
    </tr>
    <tr>
        <td>site_id</td>
        <td>a unique site identifier</td>
        <td>Data contains 16 unique sites</td>
    </tr>
    <tr>
        <td>timestamp</td>
        <td>timestamp for weather conditions</td>
        <td>we have readings for year 2016, starting from 2016-01-01 to 2016-12-31</td>
    </tr>
    <tr>
        <td>air_temperature</td>
        <td>air temperature for given site and timestamp</td>
        <td>have 55 null values, can be filled with mean or we can predict temperature using other features in train weather table</td>
    </tr>
    <tr>
        <td>cloud_coverage</td>
        <td>cloud coverage for given site and timestamp</td>
        <td>have 69173 null values, can be filled with mean or we can predict cloud coverage using other features in train weather table</td>
    </tr>
</table>

In [11]:
#FEATURE SUMMARY FOR BUILDING METADATA DATASET
feature_summary(df_building_metadata)

DataFrame shape
rows: 1449
cols: 6


,Null,Unique_Count,Data_type,Max/Min,Mean,Std,Skewness,Sample_values
site_id,0,16,int64,15/0,6.95238,5.00343,0.32549,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
building_id,0,1449,int64,1448/0,724,418.435,0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
primary_use,0,16,object,-,-,-,-,"[Education, Lodging/residential, Office, Enter..."
square_feet,0,1397,int64,875000/283,92111.8,110770,2.95077,"[7432, 2720, 5376, 23685, 116607, 8000, 27926,..."
year_built,774,117,float64,2017.0/1900.0,1967.96,31.054,-0.376338,"[2008.0, 2004.0, 1991.0, 2002.0, 1975.0, 2000...."
floor_count,1094,19,float64,26.0/1.0,3.74085,3.33368,2.23112,"[nan, 5.0, 4.0, 10.0, 6.0, 8.0, 7.0, 9.0, 13.0..."


<h3>JOINING TRAIN SET</h3>
1. First Join Train set with Building Metadata on site_id to populate building related information. Lets call joined set as train_building
2. Second Join between train_building and weather_train on site and timestamp. Lets call joined set as train_building_weather